In [1]:
%load_ext autoreload
import sys
sys.path.append("..")

In [80]:
%autoreload

import numpy as np
import pickle

import main

import darkhistory.physics as phys
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.spectools as spectools

from darkhistory.spec.spectrum import Spectrum

from darkhistory.electrons.ics.ics_spectrum import ics_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import engloss_spec

from scipy.interpolate import interp1d


In [43]:
soln = np.loadtxt(open("/Users/hongwan/GitHub/DarkHistory/darkhistory/history/recfast_standard.txt", "rb"))
xe_std = interp1d(soln[:,0], soln[:,2])
Tm_std = interp1d(soln[:,0], soln[:,1])

phys.get_optical_depth(np.flipud(soln[:,0])[4:599], np.flipud(soln[:,2])[4:599])



0.014158168060184339

# Marching Up Algorithm

In [3]:
ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf = main.load_ics_data()

********* Thomson regime scattered photon spectrum *********
Initializing...
Computing spectra by an expansion in beta...
----> Computation by expansion in beta complete!
Computing spectra by analytic series...
*** Computing series 1/12...
*** Computing series 2/12...
*** Computing series 3/12...
*** Computing series 4/12...
*** Computing series 5/12...
*** Computing series 6/12...
*** Computing series 7/12...
*** Computing series 8/12...
*** Computing series 9/12...
*** Computing series 10/12...
*** Computing series 11/12...
*** Computing series 12/12...
----> Computation by analytic series complete!
Spectrum computed!
********* Relativistic regime scattered photon spectrum *********
Initializing...
Computing series 1/4...
Computing series 2/4...
Computing series 3/4...
Computing series 4/4...
Relativistic Computation Complete!
********* Thomson regime energy loss spectrum *********
Computing nonrelativistic energy loss spectrum...
Computing energy loss spectrum by beta expansion...
C

In [81]:
def get_elec_cooling_tf_fast(
    raw_nonrel_tf, raw_rel_tf, raw_engloss_tf,
    eleceng, photeng, rs, xe, xHe=0
):

    """Returns transfer function for complete electron cooling through ICS and atomic processes.

    Parameters
    ----------
    nonrel_tf : TransFuncAtRedshift
        Raw nonrelativistic primary electron ICS transfer function.
    rel_tf : string
        Raw relativistic primary electron ICS transfer function.
    engloss_tf_filename : string
        Raw primary electron ICS energy loss transfer function.
    eleceng : ndarray
        The electron kinetic energy abscissa.
    photeng : ndarray
        The photon energy abscissa.
    rs : float
        The redshift.
    xe : float
        Free electron fraction. 
    xHe : float, optional
        Singly-ionized helium fraction, nHe+/nH. Set to nHe/nH*xe if None.
    
    Returns
    -------

    tuple of TransFuncAtRedshift
        Transfer functions for photons and low energy electrons.

    Note
    ----
    The raw transfer functions should be generated when the code package is first installed. The transfer function corresponds to the fully resolved
    photon spectrum after scattering by one electron.

    This version of the code works faster, but dispenses with energy conservation checks and several other safeguards. Use only with default abscissa, or when get_ics_cooling_tf works.

    """
    
    if xHe is None:
        xHe = xe*phys.nHe/phys.nH
        
    #####################################
    # Inverse Compton
    #####################################

    T = phys.TCMB(rs)

    # Photon transfer function for single primary electron single scattering.
    # This is dN/(dE dt), dt = 1 s.
    phot_ICS_tf = ics_spec(
        eleceng, photeng, T, nonrel_tf = raw_nonrel_tf, rel_tf = raw_rel_tf
    )

    # Downcasting speeds up np.dot
    phot_ICS_tf._grid_vals = phot_ICS_tf.grid_vals.astype('float64')

    # Energy loss transfer function for single primary electron
    # single scattering. This is dN/(dE dt), dt = 1 s.
    engloss_ICS_tf = engloss_spec(
        eleceng, photeng, T, nonrel_tf = raw_engloss_tf, rel_tf = raw_rel_tf
    )

    # Downcasting speeds up np.dot
    engloss_ICS_tf._grid_vals = engloss_ICS_tf.grid_vals.astype('float64')

    # Switch the spectra type here to type 'N'.
    if phot_ICS_tf.spec_type == 'dNdE':
        phot_ICS_tf.switch_spec_type()
    if engloss_ICS_tf.spec_type == 'dNdE':
        engloss_ICS_tf.switch_spec_type()


    # Define some useful lengths.
    N_eleceng = eleceng.size
    N_photeng = photeng.size

    # Create the secondary electron transfer functions.

    # ICS transfer function.
    elec_ICS_tf = tf.TransFuncAtRedshift(
        np.zeros((N_eleceng, N_eleceng)), in_eng = eleceng,
        rs = rs*np.ones_like(eleceng), eng = eleceng,
        dlnz = -1, spec_type = 'N'
    )

    elec_ICS_tf._grid_vals = spectools.engloss_rebin_fast(
        eleceng, photeng, engloss_ICS_tf.grid_vals, eleceng
    )
    
    # Total upscattered photon energy.
    cont_loss_ICS_vec = np.zeros_like(eleceng)
    # Deposited energy, enforces energy conservation.
    deposited_ICS_vec = np.zeros_like(eleceng)
    
    #####################
    # Excitation  
    #####################
    
    # Construct the rate matrices first. Secondary electron spectrum is an electron at in_eng - excitation energy, 
    # with a per second rate given by n*sigma*c.
    rate_matrix_exc_HI = np.diag(
        (1 - xe)*phys.nH*rs**3 * phys.coll_exc_xsec(eleceng, species='HI') * phys.c
    )
    
    rate_matrix_exc_HeI = np.diag(
        (phys.nHe/phys.nH - xHe)*phys.nH*rs**3 * phys.coll_exc_xsec(eleceng, species='HeI') * phys.c
    )
    
    rate_matrix_exc_HeII = np.diag(
        xHe*phys.nH*rs**3 * phys.coll_exc_xsec(eleceng, species='HeII') * phys.c
    )

    # Construct the TransFuncAtRedshift objects.
    # Electrons scatter from in_eng to in_eng - excitation energy.
    # Remove all of the columns (eng) that have energies below the excitation energy, 
    elec_exc_HI_tf = tf.TransFuncAtRedshift(
        np.squeeze(rate_matrix_exc_HI[:, np.where(eleceng > phys.lya_eng)]), 
        in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng[eleceng > phys.lya_eng] - phys.lya_eng,
        dlnz = -1, spec_type = 'N'
    )
    elec_exc_HeI_tf = tf.TransFuncAtRedshift(
        np.squeeze(rate_matrix_exc_HeI[:, np.where(eleceng > phys.He_exc_eng)]), 
        in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng[eleceng > phys.He_exc_eng] - phys.He_exc_eng,
        dlnz = -1, spec_type = 'N'
    )
    elec_exc_HeII_tf = tf.TransFuncAtRedshift(
        np.squeeze(rate_matrix_exc_HeII[:, np.where(eleceng > 4*phys.lya_eng)]), 
        in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng[eleceng > 4*phys.lya_eng] - 4*phys.lya_eng,
        dlnz = -1, spec_type = 'N'
    )
    
    # Rebin these transfer functions back to eleceng.
    elec_exc_HI_tf.rebin(eleceng)
    elec_exc_HeI_tf.rebin(eleceng)
    elec_exc_HeII_tf.rebin(eleceng)
   
    # Deposited energy for excitation.
    deposited_exc_vec = np.zeros_like(eleceng)
    
    #####################
    # Ionization  
    #####################
    
    # Construct the rate vector first. Secondary electron spectrum is an electron at in_eng - excitation energy, 
    # with a per second rate given by n*sigma*c.
    rate_vec_ion_HI = (
        (1 - xe)*phys.nH*rs**3 * phys.coll_ion_xsec(eleceng, species='HI') * phys.c
    )
    
    rate_vec_ion_HeI = (
        (phys.nHe/phys.nH - xHe)*phys.nH*rs**3 * phys.coll_ion_xsec(eleceng, species='HeI') * phys.c
    )
    
    rate_vec_ion_HeII = (
        xHe*phys.nH*rs**3 * phys.coll_ion_xsec(eleceng, species='HeII') * phys.c
    )
    
    # Construct the spectra. 
    elec_spec_ion_HI = np.array(
        [rate*phys.coll_ion_sec_elec_spec(in_eng, eleceng, species='HI') for rate,in_eng in zip(rate_vec_ion_HI,eleceng)]
    )
    elec_spec_ion_HeI = np.array(
        [rate*phys.coll_ion_sec_elec_spec(in_eng, eleceng, species='HeI') for rate,in_eng in zip(rate_vec_ion_HeI,eleceng)]
    )
    elec_spec_ion_HeII = np.array(
        [rate*phys.coll_ion_sec_elec_spec(in_eng, eleceng, species='HeII') for rate,in_eng in zip(rate_vec_ion_HeII,eleceng)]
    )
    
#     for i in np.arange(500):
# #         print(np.dot(elec_spec_ion_HI[i], eleceng) + rate_vec_ion_HI[i]*phys.rydberg - eleceng[i]*rate_vec_ion_HI[i])
#         print('Injection energy: ', eleceng[i])
#         print(np.dot(elec_spec_ion_HeI[i], eleceng))
#         print(rate_vec_ion_HeI[i]*phys.He_ion_eng)
#         print(eleceng[i]*rate_vec_ion_HeI[i])
#         print(np.dot(elec_spec_ion_HeI[i], eleceng) + rate_vec_ion_HeI[i]*phys.He_ion_eng - eleceng[i]*rate_vec_ion_HeI[i])
    
    
    
    # Construct the TransFuncAtRedshift objects.
    # Electrons scatter from in_eng to in_eng - excitation energy.
    # Remove all of the columns (eng) that have energies below the excitation energy, 
    elec_ion_HI_tf = tf.TransFuncAtRedshift(
        elec_spec_ion_HI, in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng, dlnz = -1, spec_type = 'N'
    )
    elec_ion_HeI_tf = tf.TransFuncAtRedshift(
        elec_spec_ion_HeI, in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng, dlnz = -1, spec_type = 'N'
    )
    elec_ion_HeII_tf = tf.TransFuncAtRedshift(
        elec_spec_ion_HeII, in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng, dlnz = -1, spec_type = 'N'
    )
    
    # Deposited energy for ionization.
    deposited_ion_vec = np.zeros_like(eleceng)
    
    #############################################
    # Heating
    #############################################
    
    dE_heat_dt = phys.elec_heating_engloss_rate(eleceng, xe, rs)
    
    deposited_heat_vec = np.zeros_like(eleceng)
    
    
    #############################################
    # Initialization of secondary spectra 
    #############################################

    # Low and high energy boundaries
    loweng = 3000
    eleceng_high = eleceng[eleceng > loweng]
    eleceng_high_ind = np.arange(eleceng.size)[eleceng > loweng]
    eleceng_low = eleceng[eleceng <= loweng]
    eleceng_low_ind  = np.arange(eleceng.size)[eleceng <= loweng]


    if eleceng_low.size == 0:
        raise TypeError('Energy abscissa must contain a low energy bin below 3 keV.')

    # Empty containers for quantities.
    # Final secondary photon spectrum.
    sec_phot_tf = tf.TransFuncAtRedshift(
        np.zeros((N_eleceng, N_photeng)), in_eng = eleceng,
        rs = rs*np.ones_like(eleceng), eng = photeng,
        dlnz = -1, spec_type = 'N'
    )
    # Final secondary low energy electron spectrum.
    sec_lowengelec_tf = tf.TransFuncAtRedshift(
        np.zeros((N_eleceng, N_eleceng)), in_eng = eleceng,
        rs = rs*np.ones_like(eleceng), eng = eleceng,
        dlnz = -1, spec_type = 'N'
    )

    # Start building sec_phot_tf and sec_lowengelec_tf.
    # Low energy regime first.

    sec_lowengelec_tf._grid_vals[:eleceng_low.size, :eleceng_low.size] = (
        np.identity(eleceng_low.size)
    )

    # Continuum energy loss rate per electron, dU_CMB/dt.
    CMB_upscatter_eng_rate = phys.thomson_xsec*phys.c*phys.CMB_eng_density(T)


    # High energy electron loop to get fully resolved spectrum.
    for i, eng in zip(eleceng_high_ind, eleceng_high):
        

        # print('Check energies and indexing: ')
        # print(i, eleceng[i], eng)

        phot_ICS_N = phot_ICS_tf.grid_vals[i]
        
        elec_ICS_N      = elec_ICS_tf.grid_vals[i]
        
        elec_exc_HI_N   = elec_exc_HI_tf.grid_vals[i]
        elec_exc_HeI_N  = elec_exc_HeI_tf.grid_vals[i]
        elec_exc_HeII_N = elec_exc_HeII_tf.grid_vals[i]
        
        elec_ion_HI_N   = elec_ion_HI_tf.grid_vals[i]
        elec_ion_HeI_N  = elec_ion_HeI_tf.grid_vals[i]
        elec_ion_HeII_N = elec_ion_HeII_tf.grid_vals[i]
        
        elec_heat_spec = spectools.rebin_N_arr(np.array([1]), np.array([eng]), eleceng)
        elec_heat_spec.eng -= dE_heat_dt[i]
        elec_heat_spec.rebin(eleceng)
        elec_heat_N = elec_heat_spec.N
        
        
        sec_elec_spec_N = (
            elec_ICS_N 
            + elec_exc_HI_N + elec_exc_HeI_N + elec_exc_HeII_N
            + elec_ion_HI_N + elec_ion_HeI_N + elec_ion_HeII_N
            + elec_heat_N
        )
        sec_phot_spec_N = phot_ICS_N
        

        sec_elec_totN = np.sum(sec_elec_spec_N)
        # The *net* total energy of secondary electrons produced
        # per unit time.
        sec_elec_toteng = np.dot(sec_elec_spec_N, eleceng)
        # The total energy of secondary photons produced per unit time.
        sec_phot_toteng = np.dot(sec_phot_spec_N, photeng)
        # Deposited ICS energy per unit time, dD/dt.
        # Numerical error (should be zero except for numerics)
        deposited_ICS_eng = (
            np.sum(elec_ICS_N)*eng - np.dot(elec_ICS_N, eleceng)
            - (np.dot(phot_ICS_N, photeng) - CMB_upscatter_eng_rate)
        )
        # Deposited excitation energy. 
        deposited_exc_eng = (
            phys.lya_eng*np.sum(elec_exc_HI_N)
            + phys.He_exc_eng*np.sum(elec_exc_HeI_N)
            + 4*phys.lya_eng*np.sum(elec_exc_HeII_N)
        )
        # Deposited ionization energy. Remember that the secondary spectrum
        # has two electrons for each ionization event.
        deposited_ion_eng = (
            phys.rydberg*np.sum(elec_ion_HI_N/2)
            + phys.He_ion_eng*np.sum(elec_ion_HeI_N/2)
            + 4*phys.rydberg*np.sum(elec_ion_HeII_N/2)
        )
        # Deposited heating energy. 
        deposited_heat_eng = dE_heat_dt[i]
        
        # In the original code, the energy of the electron has gamma > 20,
        # then the continuum energy loss is assigned to deposited_eng instead.
        # I'm not sure if this is necessary, but let's be consistent with the
        # original code for now.

        continuum_engloss = CMB_upscatter_eng_rate
        
        if eng + phys.me > 20*phys.me:
            deposited_ICS_eng += CMB_upscatter_eng_rate
            continuum_engloss = 0

        # Remove self-scattering.
        sec_elec_spec_N[i] = 0
        
        # Rescale.
        toteng_no_self_scatter = (
            np.dot(sec_elec_spec_N, eleceng)
            + np.dot(sec_phot_spec_N, photeng)
            - continuum_engloss
            + deposited_ICS_eng
            + deposited_exc_eng
            + deposited_ion_eng
            + deposited_heat_eng
        )
        
        fac = eng/toteng_no_self_scatter
        # Normalize to one electron. 
        
        sec_elec_spec_N    *= fac
        sec_phot_spec_N    *= fac
        continuum_engloss  *= fac
        deposited_ICS_eng  *= fac
        deposited_exc_eng  *= fac
        deposited_ion_eng  *= fac
        deposited_heat_eng *= fac

        # Get the full secondary photon spectrum. Type 'N'
        resolved_phot_spec_vals = np.dot(
            sec_elec_spec_N, sec_phot_tf.grid_vals
        )
        
        # Get the full secondary low energy electron spectrum. Type 'N'.

        resolved_lowengelec_spec_vals = np.dot(
            sec_elec_spec_N, sec_lowengelec_tf.grid_vals
        )

        # The resolved lowengelec spectrum is simply one electron
        # in the bin just below 3 keV.
        # Added directly to sec_lowengelec_tf. Removed the dot for speed.
        # resolved_lowengelec_spec_vals = np.zeros_like(eleceng)
        # resolved_lowengelec_spec_vals[eleceng_low_ind[-1]] += 1

        # Add the resolved spectrum to the first scatter.
        sec_phot_spec_N += resolved_phot_spec_vals

        # Resolve the secondary electron continuum loss and deposition.
        continuum_engloss += np.dot(sec_elec_spec_N, cont_loss_ICS_vec)

        deposited_ICS_eng  += np.dot(sec_elec_spec_N, deposited_ICS_vec)
        deposited_exc_eng  += np.dot(sec_elec_spec_N, deposited_exc_vec)
        deposited_ion_eng  += np.dot(sec_elec_spec_N, deposited_ion_vec)
        deposited_heat_eng += np.dot(sec_elec_spec_N, deposited_heat_vec)

        # Now, append the resulting spectrum to the transfer function.
        # Do this without calling append of course: just add to the zeros
        # that fill the current row in _grid_vals.
        sec_phot_tf._grid_vals[i] += sec_phot_spec_N
        sec_lowengelec_tf._grid_vals[i] += resolved_lowengelec_spec_vals
        # Set the correct values in cont_loss_vec and deposited_vec.
        
        cont_loss_ICS_vec[i]  = continuum_engloss
        deposited_ICS_vec[i]  = deposited_ICS_eng
        deposited_exc_vec[i]  = deposited_exc_eng
        deposited_ion_vec[i]  = deposited_ion_eng
        deposited_heat_vec[i] = deposited_heat_eng

        
        check = True
        verbose = True
        failed_conservation_check = False
        
        if check:

            conservation_check = (
                eng
                - np.dot(sec_lowengelec_tf.grid_vals[i], eleceng)
                + cont_loss_ICS_vec[i]
                - np.dot(sec_phot_tf.grid_vals[i], photeng)
                - deposited_exc_vec[i]
                - deposited_ion_vec[i]
                - deposited_heat_vec[i]
            )

            if conservation_check/eng > 0.1:
                failed_conservation_check = True
                
            if verbose or failed_conservation_check:
                
                print('***************************************************')
                print('rs: ', rs)
                print('injected energy: ', eng)
                print(
                    'Energy in low energy electrons: ',
                    np.dot(sec_lowengelec_tf.grid_vals[i], eleceng)
                )
                print('Energy in photons: ', np.dot(sec_phot_tf.grid_vals[i], photeng))
                print('Continuum_engloss: ', cont_loss_ICS_vec[i])
                print(
                    'Energy in photons - Continuum: ',
                    np.dot(sec_phot_tf.grid_vals[i], photeng) - cont_loss_ICS_vec[i]
                )
                print(
                    'Deposited in excitation: ', deposited_exc_vec[i]
                )
                print(
                    'Deposited in ionization: ', deposited_ion_vec[i]
                )
                print(
                    'Deposited in heating: ', deposited_heat_vec[i]
                )
                print(
                    'Energy is conserved up to (%): ',
                    conservation_check/eng*100
                )
                print('Deposited in ICS: ', deposited_ICS_vec[i])
                print(
                    'Energy conservation with deposited (%): ',
                    (conservation_check - deposited_ICS_vec[i])/eng*100
                )
                print('***************************************************')
                
            if failed_conservation_check:
                raise RuntimeError('Conservation of energy failed.')


    return (sec_phot_tf, sec_lowengelec_tf, cont_loss_ICS_vec)


In [88]:
eleceng = 10**np.arange(0, np.log10(5e12), 0.0254)
photeng = 10**np.arange(-4, np.log10(5e12), 0.0334)


rs = 600
xe = 1e-2

get_elec_cooling_tf_fast(
    ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf,
    eleceng, photeng, rs, xe, xHe=0
)

***************************************************
rs:  600
injected energy:  3018.5613014197993
Energy in low energy electrons:  2932.9121153799
Energy in photons:  456.11057080314566
Continuum_engloss:  449.0060332030339
Energy in photons - Continuum:  7.104537600111769
Deposited in excitation:  13.980782285783445
Deposited in ionization:  63.29227567424261
Deposited in heating:  1.2821192089994786
Energy is conserved up to (%):  -0.00034879958320461953
Deposited in ICS:  -0.010528729238503277
Energy conservation with deposited (%):  1.2425735410850618e-14
***************************************************
***************************************************
rs:  600
injected energy:  3200.368594586517
Energy in low energy electrons:  3029.8452693326453
Energy in photons:  928.6562974127817
Continuum_engloss:  913.7260998386595
Energy in photons - Continuum:  14.93019757412219
Deposited in excitation:  27.727370558609095
Deposited in ionization:  125.31593613880662
Deposited in heat

Deposited in heating:  182.08985776470735
Energy is conserved up to (%):  -0.009667753713259427
Deposited in ICS:  -3.027788800186759
Energy conservation with deposited (%):  -8.828771227776224e-14
***************************************************
***************************************************
rs:  600
injected energy:  33204.73357975065
Energy in low energy electrons:  12705.338334538726
Energy in photons:  154197.14180473107
Continuum_engloss:  140043.47616702013
Energy in photons - Continuum:  14153.665637710947
Deposited in excitation:  1175.8551215369284
Deposited in ionization:  4983.178366253939
Deposited in heating:  189.90768971990386
Energy is conserved up to (%):  -0.009672024628891024
Deposited in ICS:  -3.2115700097426925
Energy conservation with deposited (%):  -1.4587989525064373e-13
***************************************************
***************************************************
rs:  600
injected energy:  35204.64748894213
Energy in low energy electrons:  1

***************************************************
rs:  600
injected energy:  203516.67563669104
Energy in low energy electrons:  17569.636313419374
Energy in photons:  604005.2232503478
Continuum_engloss:  427521.2149586485
Energy in photons - Continuum:  176484.00829169928
Deposited in excitation:  1763.03660621542
Deposited in ionization:  7322.045759637112
Deposited in heating:  387.74239403518555
Energy is conserved up to (%):  -0.004812248571131314
Deposited in ICS:  -9.793728315280317
Energy conservation with deposited (%):  -2.962825170212764e-14
***************************************************
***************************************************
rs:  600
injected energy:  215774.44091526646
Energy in low energy electrons:  17629.179563025144
Energy in photons:  625126.8609199638
Continuum_engloss:  436484.77361687314
Energy in photons - Continuum:  188642.08730309067
Deposited in excitation:  1770.4243102972516
Deposited in ionization:  7350.5587036263105
Deposited in heati

Continuum_engloss:  616537.3265305167
Energy in photons - Continuum:  849912.4656670535
Deposited in excitation:  1856.1231304623686
Deposited in ionization:  7677.1714300630865
Deposited in heating:  470.07958080685273
Energy is conserved up to (%):  -0.0016088319193560282
Deposited in ICS:  -14.128975566805094
Energy conservation with deposited (%):  -2.4769305413794202e-14
***************************************************
***************************************************
rs:  600
injected energy:  931107.875467829
Energy in low energy electrons:  18321.385876986336
Energy in photons:  1525031.1007379093
Continuum_engloss:  622241.8970038167
Energy in photons - Continuum:  902789.2037340926
Deposited in excitation:  1857.3800927854622
Deposited in ionization:  7681.868038114985
Deposited in heating:  472.29736817765405
Energy is conserved up to (%):  -0.0015314704884052267
Deposited in ICS:  -14.259642327665281
Energy conservation with deposited (%):  -3.6667121442901605e-14
****

***************************************************
rs:  600
injected energy:  5076916.258882484
Energy in low energy electrons:  18419.182275850828
Energy in photons:  5772556.511681149
Continuum_engloss:  724152.5401345982
Energy in photons - Continuum:  5048403.97154655
Deposited in excitation:  1870.0288638077684
Deposited in ionization:  7728.653683833893
Deposited in heating:  511.02108566473134
Energy is conserved up to (%):  -0.0003269420328628224
Deposited in ICS:  -16.59857322000878
Energy conservation with deposited (%):  -6.942734237217688e-14
***************************************************
***************************************************
rs:  600
injected energy:  5382697.825162881
Energy in low energy electrons:  18419.759839278406
Energy in photons:  6080074.1502618
Continuum_engloss:  725890.1623349141
Energy in photons - Continuum:  5354183.9879268855
Deposited in excitation:  1870.1051440944486
Deposited in ionization:  7728.929981622774
Deposited in heating:  

...

Energy conservation with deposited (%):  1.0450422194519567e-14
***************************************************
***************************************************
rs:  600
injected energy:  2358305153.2776523
Energy in low energy electrons:  18391.772243632942
Energy in photons:  2358978316.253265
Continuum_engloss:  737862.4520799214
Energy in photons - Continuum:  2358240453.801185
Deposited in excitation:  1867.4246955688411
Deposited in ionization:  7717.4855319336075
Deposited in heating:  522.944286562418
Energy is conserved up to (%):  0.0015349943012784486
Deposited in ICS:  36199.84970896385
Energy conservation with deposited (%):  2.5615893345829625e-14
***************************************************
***************************************************
rs:  600
injected energy:  2500345361.6964297
Energy in low energy electrons:  18391.77218052467
Energy in photons:  2501018515.4846926
Continuum_engloss:  737862.4493746299
Energy in photons - Continuum:  2500280653.03

Continuum_engloss:  737862.4273381307
Energy in photons - Continuum:  21767006458.761223
Deposited in excitation:  1867.4246387267297
Deposited in ionization:  7717.485292514535
Deposited in heating:  522.9773339819174
Energy is conserved up to (%):  0.00016699462301055944
Deposited in ICS:  36349.83867133117
Energy conservation with deposited (%):  1.5404241845103344e-15
***************************************************
***************************************************
rs:  600
injected energy:  23078097296.983982
Energy in low energy electrons:  18391.77166636652
Energy in photons:  23078770308.712166
Continuum_engloss:  737862.4273006564
Energy in photons - Continuum:  23078032446.284866
Deposited in excitation:  1867.4246386414147
Deposited in ionization:  7717.485292154106
Deposited in heating:  522.9776071307405
Energy is conserved up to (%):  0.00015751315822339074
Deposited in ICS:  36351.039913736255
Energy conservation with deposited (%):  -1.4688431204552337e-14
********

***************************************************
rs:  600
injected energy:  149968483550.2374
Energy in low energy electrons:  18391.771658947167
Energy in photons:  149969156541.729
Continuum_engloss:  737862.4269799821
Energy in photons - Continuum:  149968418679.30203
Deposited in excitation:  1867.4246379134222
Deposited in ionization:  7717.485289075677
Deposited in heating:  522.9822840477106
Energy is conserved up to (%):  2.425261003127258e-05
Deposited in ICS:  36371.27151747653
Energy conservation with deposited (%):  -2.1487371338157917e-14
***************************************************
***************************************************
rs:  600
injected energy:  159001052821.4863
Energy in low energy electrons:  18391.771658919242
Energy in photons:  159001725812.64972
Continuum_engloss:  737862.4269787692
Energy in photons - Continuum:  159000987950.22275
Deposited in excitation:  1867.4246379106917
Deposited in ionization:  7717.4852890641
Deposited in heating:  

***************************************************
rs:  600
injected energy:  919178801694.4913
Energy in low energy electrons:  18391.77165866243
Energy in photons:  919179474678.2498
Continuum_engloss:  737862.4269675901
Energy in photons - Continuum:  919178736815.8228
Deposited in excitation:  1867.4246378856253
Deposited in ionization:  7717.485288957663
Deposited in heating:  522.9841516764325
Energy is conserved up to (%):  3.957772179588173e-06
Deposited in ICS:  36379.00266070191
Energy conservation with deposited (%):  2.539599620760753e-14
***************************************************
***************************************************
rs:  600
injected energy:  974540741766.3053
Energy in low energy electrons:  18391.77165866105
Energy in photons:  974541414749.8528
Continuum_engloss:  737862.4269675297
Energy in photons - Continuum:  974540676887.4258
Deposited in excitation:  1867.4246378854916
Deposited in ionization:  7717.485288957093
Deposited in heating:  522.

(<darkhistory.spec.transferfunction.TransFuncAtRedshift at 0x111dd8a20>,
 array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000

In [ ]:
j=89
print(np.sum(phys.coll_ion_sec_elec_spec(eleceng[j], eleceng, species='HeI')))
print(eleceng[j] - np.dot(phys.coll_ion_sec_elec_spec(eleceng[j], eleceng, species='HeI'), eleceng))
    

In [ ]:
a = 1
a.shape

In [77]:
eps_0 = 8.85418782e-12
phys.ele**4/((4*np.pi*eps_0)**2*phys.me/phys.c**2)*(100**2/phys.ele**2)/phys.c

1.2164780860739776e-09